### Preserving the Wrapped Function

The techniques in this chapter for creating decorators are time-tested, and valuable in many situations. But the resulting decorators have a few problems:

- Function objects automatically have certain attributes, like 
    - \_\_name\_\_ 
    - \_\_doc\_\_
    - \_\_module\_\_ 
    - etc.  

<br>

- Decorators interfere with introspection - masking the wrapped function’s signature, and blocking inspect.getsource().
<br><br>

- Decorators cannot be applied in certain more exotic situations - like class methods, or descriptors - without going through some heroic contortions.



### First problem... : functools

In [2]:
import functools
def printlog(func):
    #@functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('CALLING: {}'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapper

@printlog
def foo(x):
    print(x+2)
    
foo.__name__

'wrapper'

In [4]:
import functools
def printlog(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('CALLING: {}'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapper

@printlog
def foo(x):
    print(x+2)
    
foo.__name__

'foo'

When applied to the wrapper function, it essentially copies certain attributes from the wrapped function to the wrapper. It is equivalent to this:

In [ ]:
def printlog(func):
    def wrapper(*args, **kwargs):
        print('CALLING: {}'.format(func.__name__))
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    wrapper.__annotations__ = func.__annotations__
    return wrapper

functools.wraps is a actually a convenient shortcut of the more general update_wrapper. 


Since **functools.wraps only works with function-based decorators**, your class-based decorators must use update_wrapper instead:

In [ ]:
import functools
class PrintLog:
    def __init__(self, func):
        self.func = func
        functools.update_wrapper(self, func)
        
    def __call__(self, *args, **kwargs):
        print('CALLING: {}'.format(self.func.__name__))
        return self.func(*args, **kwargs)

While useful for copying over \_\_name\_\_, \_\_doc\_\_, and the other attributes
- wraps 
- update_wrapper 

do not help with the other problems mentioned above.

So what should you do in practice?